# Шаблоны jinja2

Шаблонизаторы - это f-строки на стероидах.

## Источники

* [Официальный сайт](https://pypi.org/project/Jinja2/)
* [Документация](https://jinja.palletsprojects.com/en/3.1.x/)
* [Основы jinja](https://pythonru.com/uroki/7-osnovy-shablonizatora-jinja) - на русском, урок по jinja из курса по Flask.
* [RealPython](https://realpython.com/primer-on-jinja-templating/) (пересказ на русском [тут](https://proglib.io/p/rukovodstvo-dlya-nachinayushchih-po-shablonam-jinja-v-flask-2022-09-05))
* [Tutorial](https://ttl255.com/jinja2-tutorial-part-1-introduction-and-variable-substitution/) - полезно devops (там про интеграцию с ansible)
* [zetcode](https://zetcode.com/python/jinja/) - вот отсюда урок во многом взят
* [wiki](https://lectureswww.readthedocs.io/6.www.sync/2.codding/3.templates/jinja2.html) - Ru
* [Tutorial](https://dvmn.org/encyclopedia/modules/jinja2/) - Ru (пример страницы сайта с кепками)

## Установка

`pip3 install jinja2`

## Hello, world!

In [5]:
from jinja2 import Template

template = Template('Hello {{ name }}!')
template.render(name='Вася')

'Hello Вася!'

## Синтаксические конструкции

* `{{ }}` - expressions to print to the template output - выражение, которое будет подставляться в шаблон
* `{# Комментарии #}`
* `{% код %}` - statements - управляющие конструкции
* `# ##` - line statements 


## `{{ выражение }}` - подстановки переменных

Переменные можно подставить из 

* именованныx аргументы функции render
* словаря
* атрибутов экземпляра класса

### Именованные аргументы

In [6]:
from jinja2 import Template

name = 'Peter'
age = 34

tm = Template("My name is {{ name }} and I am {{ age }}")
msg = tm.render(age=age, name=name)

print(msg)

My name is Peter and I am 34


### Словарь

In [7]:
from jinja2 import Template

d = {
    'name': 'Peter',
    'age': 34
}
tm = Template("My name is {{ person.name }} and I am {{ person.age }}")
msg = tm.render(person=d)
print(msg)

tm = Template("My name is {{ person['name'] }} and I am {{ person['age'] }}")
msg = tm.render(person=d)
print(msg)

My name is Peter and I am 34
My name is Peter and I am 34


#### Вложенный словарь

In [11]:
from jinja2 import Template

d = {
    'name': 'Peter',
    'age': 34,
    'pet': {
        'name': 'Barsik',
        'kind': 'cat'
    }
}
template = """
My name is {{ person.name }} and I am {{ person.age }}.
My {{ person.pet.kind }}'s name is {{ person.pet.name}}.
"""
tm = Template(template)
msg = tm.render(person=d)
print(msg)

print(d['name'])
# print(d.name)


My name is Peter and I am 34.
My cat's name is Barsik.
Peter


### Класс

In [1]:
from jinja2 import Template
from dataclasses import dataclass

@dataclass
class Person:
    name: str
    age: int
    def salary(self):
        return 100000

p1 = Person('Peter', 34)
tm = Template("""
My name is {{ person.name }} and I am {{ person.age }} 
with salary {{ person.salary() }} dollars.""")          # вызов метода объекта
msg = tm.render(person=p1)
print(msg)



My name is Peter and I am 34 
with salary 100000 dollars.


## Экранирование

### rawdata

In [19]:
from jinja2 import Template

data = '''
{% raw %}
His name is {{ name }}
{% endraw %}
'''

tm = Template(data)
msg = tm.render(name='Peter')

print(msg)



His name is {{ name }}



### escape

In [2]:
from jinja2 import Template #, escape

data = '<a>Today is a sunny day x < 23 пример </a>'

tm = Template("{{ data | e}}")
msg = tm.render(data=data)

print(msg)
#print(escape(data))

&lt;a&gt;Today is a sunny day x &lt; 23 пример &lt;/a&gt;


## Шаблон во внешнем файле

Файл templates/message.txt

Создадим файлы писем студентам о пройденном экзамене.

In [3]:
# write_messages.py

from jinja2 import Environment, FileSystemLoader

max_score = 100
test_name = "Python Challenge"
students = [
    {"name": "Sandrine",  "score": 100},
    {"name": "Gergeley", "score": 87},
    {"name": "Frieda", "score": 92},
]

environment = Environment(loader=FileSystemLoader("templates/"))
template = environment.get_template("message.txt")

for student in students:
    filename = f"mails/message_{student['name'].lower()}.txt"
    content = template.render(
        student,
        max_score=max_score,
        test_name=test_name
    )
    with open(filename, mode="w", encoding="utf-8") as message:
        message.write(content)
        print(f"... wrote {filename}")

... wrote mails/message_sandrine.txt
... wrote mails/message_gergeley.txt
... wrote mails/message_frieda.txt


## Песочница

Создадим тут песочницу для исследования шаблонов

In [5]:
text = "My name is {{ name }} and I am {{ age }}"

data = {'name':'Peter', 'age':34}
Template(text).render(**data)

'My name is Peter and I am 34'

## set variable

Можно определять переменные и их значения прямо в шаблоне. Чем мы будем пользоваться в наших примерах.

Переменные в шаблоне нужны, чтобы запомнить результат сложной операции для дальнейшего использования. По возможности считаем в python, а не в jinja.

Переменные, определенные вне управляющих конструкций (о них дальше), ведут себя как **глобальные переменные** и доступны внутри любой структуры. 

Переменные, созданные внутри конструкций, ведут себя как **локальные переменные** и видимы только внутри этих конкретных конструкций. Единственное исключение — инструкция if.

## if else endif

Изменим шаблон, чтобы если студент не набрал должного количества баллов, мы ему написали, что тест не пройден.

In [9]:
text = '''
{% if bookmarks %}
    <p>User has some bookmarks</p>
{% else %}
    <p>No bookmarks</p>
{% endif %}
'''

print(Template(text).render(bookmarks=['one', 'two']))
print('-----')
print(Template(text).render(bookmarks=[]))    



    <p>User has some bookmarks</p>

-----


    <p>No bookmarks</p>



Используйте конструкцию так, как вы привыкли в python:

* `if`..`elif`..`else` + `endif`
* вложенные условия (отступы не обязательны, но повышают читаемость)
* однострочники
* операторы сравнения, in, логические операторы - пишем как в python

## Undefined переменная

Если переменная из шаблона не определена, то никакой диагностики. Просто пустая строка.

In [10]:
text = "My name is {{ name }} and I am {{ age }}"

data = {'gender':'male', 'age':34}
Template(text).render(**data)

'My name is  and I am 34'

### Определим значение по умолчанию в шаблоне

Значение по умолчанию в шаблоне

### Диагностика undefined и пустого значения

Официально рекомендованное решение:

Проверка, что определена и не пуста (т.е bool(variable) не False)

Однострочник ОК или N/A для переменной из [официальной документации](https://jinja.palletsprojects.com/en/3.1.x/templates/#if-expression)

## for .. endfor - перебираем последовательность

### Список

In [6]:
text = '''
{% set user_list = ['tom', 'jerry', 'spike'] %}

<ul>
{% for user in user_list %}
    <li>{{ user }}</li>
{% endfor %}
</ul>'''

print(Template(text).render())
Template(text).render()




<ul>

    <li>tom</li>

    <li>jerry</li>

    <li>spike</li>

</ul>


'\n\n\n<ul>\n\n    <li>tom</li>\n\n    <li>jerry</li>\n\n    <li>spike</li>\n\n</ul>'

### Словарь

In [19]:
text = '''
{% set employee = { 'name': 'tom', 'age': 25, 'designation': 'Manager' } %}

<ul>
{% for key in employee %}
<li>{{ key }} : {{ employee[key] }}</li>
{% endfor %}
</ul>'''

print(Template(text).render())




<ul>

<li>name : tom</li>

<li>age : 25</li>

<li>designation : Manager</li>

</ul>


In [12]:
text = '''
{% set employee = { 'name': 'tom', 'age': 25, 'designation': 'Manager' } %}

<ul>
{% for key, value in employee.items() %}
<li>{{ key }} : {{ value }}</li>
{% endfor %}
</ul>'''

print(Template(text).render())




<ul>

<li>name : tom</li>

<li>age : 25</li>

<li>designation : Manager</li>

</ul>


### for..else..endfor - когда последовательность пустая

else - в python и jinja условия попадания в часть else разные.

* python: нет break
* jinja: **пустая последовательность** 

In [13]:
text = '''
{% set user_list = [] %}

<ul>
{% for user in user_list %}
    <li>{{ user }}</li>
{% else %}
    <li>user_list is empty</li>
{% endfor %}
</ul>'''

print(Template(text).render())




<ul>

    <li>user_list is empty</li>

</ul>


Вложенные циклы можно написать.

### Переменная loop

Специальная переменная для отслеживания прогресса цикла.

In [22]:
text = '''
{% set user_list = ['Alex', 'Bob', 'Charley'] %}

<ul>
{% for user in user_list %}
    <li>{{ loop.index }} - {{ user }}</li>
{% endfor %}
</ul>'''

print(Template(text).render())




<ul>

    <li>1 - Alex</li>

    <li>2 - Bob</li>

    <li>3 - Charley</li>

</ul>


| Метод | Значение |
|----|----|
| loop.index | Счетчик итерации цикла, **начинается с 1** |
| loop.index0 | Счетчик итерации цикла, **начинается с 0** |
| loop.revindex | Номер итерации цикла с конца, **начинается с 1** |
| loop.revindex0 | Номер итерации цикла с конца, **начинается с 0** |
| loop.first | True, если первая итерация, иначе False |
| loop.last | True, если последняя итерация, иначе False |
| loop.length | возвращает длину цикла(количество итераций) |

Больше переменных в [документации](https://jinja.palletsprojects.com/en/3.0.x/templates/#for)


## Фильтры

Функции, которые применяются к данным. Их можно применять цепочкой один за другим.

`variable_or_value|filter_name`

Применить к переменной comment функцию title (это str.title)

Сначала  striptags (удалить все html теги), потом title.

Фильтры с аргументами, округлим числа до 2 десятичных знаков после запятой.

| Фильтр | Что делает |
|----|----|
| upper | Все символы в верхнем регистре |
| lower | Все символы в нижнем регистре |
| capitalize | делает заглавной первую букву и приводит остальные к нижнему регистру |
| escape |  экранирует значение |
| safe | предотвращает экранирование |
| length | возвращает количество элементов в последовательности |
| trim | удаляет пустые символы в начале и в конце |
| random | возвращает случайный элемент последовательности |ости

[Документация](https://jinja.pocoo.org/docs/2.10/templates/#list-of-builtin-filters)

## Макросы

* Похожи на функции в python.
* Определение макроса должно быть до первого вызова.
* Обычно макросы хранят в отдельном шаблоне и импортируют, когда нужны.

Макрос с обязательным аргументом `post_list` и необязательным `sep`:

Вызов макроса:

Если сохранили в отдельном файле `macros.html` и импортируем, то импорт и вызов так:

или с `from`

### varags и kwargs

Аналогично **args** и **kwargs** в python, в jinja можно передавать произвольное количество позиционных и именованных аргументов с помощью **varags** и **kwargs**

**В аргументах макроса их определять не нужно, они всегда там есть.**

Дополнительный позиционный аргумент, "apple", присваивается varargs, а дополнительные аргументы-ключевые слова (name='spike', age=15) в kwargs.

In [7]:
text = '''
{% macro custom_renderer(para) %}
    <p>{{ para }}</p>
    <p>varargs: {{ varargs }}</p>
    <p>kwargs: {{ kwargs }}</p>
{%  endmacro  %}

{{ custom_renderer("some content", "apple", name='spike', age=15) }}
'''

print(Template(text).render())





    <p>some content</p>
    <p>varargs: ('apple',)</p>
    <p>kwargs: {'name': 'spike', 'age': 15}</p>



## Экранирование

Символы < и > заменяются на &lt; и &gt; и прочие замены [html символов](https://www.toptal.com/designers/htmlarrows/symbols/)

In [14]:
data = "<a>Today's is a sunny day</a>"

Template("{{ data }}").render(data=data)

"<a>Today's is a sunny day</a>"

In [29]:
data = '<a>Today is a sunny day</a>'

Template("{{ data | escape }}").render(data=data)

'&lt;a&gt;Today is a sunny day&lt;/a&gt;'

In [30]:
data = '<a>Today is a sunny day</a>'

Template("{{ data | safe }}").render(data=data)

'<a>Today is a sunny day</a>'

Если кода много, используйте оператор `autoescape`, он принимает значения `true` и `false` 

Можно экранировать отдельные части кода, совмещая оператор autoescape и фильтр escape:

## Вложенные шаблоны

**include** - рендерит шаблон внутри другого шаблона (не путать с макросами!)

Есть раздел, который повторяется в разных местах сайта. Лежит в `templates/nav.html`

Добавим содержимое этого шаблона в `home.html`:

получим

## Наследование шаблонов

Похоже на наследование классов, те же принципы ООП.

**Базовый шаблон** содержит в себе скелет html и отдельные **маркеры**, которые дочерние шаблоны будут переопределять.

* **block** .. **endblock** - инструкция создания маркера.
    * аргумент - имя блока (маркер) - должно быть уникальным в рамках шаблона.
* **extends** - расширение (наследование) основного шаблона.

Базовый шаблон в файле templates/base.html с блоками title, nav, content (сейчас пуст).

**Дочерний шаблон** - расширяет базовый шаблон.

* extends:
    * загружается базовый шаблон, то есть base.html,
    * затем блоки заменяются на описанные в дочернем шаблоне,
    * если блок в дочернем шаблоне не найден, остается блок из базового шаблона.
 
То есть блоки title и nav остаются, блок content переписывается.

получим

Можем переписать два блока, title и contents:

### super() - взяли содержимое этого блока базового шаблона

Оставим пункты блока навигации, добавив к ним новые пункты с помощью super.

получим:

# Заключение

* Добивайтесь равновесия между "закодить в python" и "закодить в шаблонах".
* Ищите примеры шаблонов.
    * https://github.com/tatyderb/ejudge_tools/blob/master/ej_plot_contest/jinja_templates/result_table_template.html